In [1]:
#Library 예시

import math
import time
from itertools import chain
import argparse
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.optim import Optimizer, Adam
from torch.optim.lr_scheduler import LambdaLR
# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
#device=torch.device('cpu')
torch.cuda.is_available()

True

In [2]:
wavenumbers=np.linspace(1000,8000,351)

In [4]:
# 별도의 데이터 Pre-Processing 과정은 없고 모델 훈련시 검증을 위해 train 중 54000개를 validation 용으로 분리.
# 새로 만든 train.csv는 train1.csv, validation은 val.csv로 저장.
# dataframe.sample(frac=1) 을 통해 셔플.
path_train = 'train_parameter.csv'
#path_test = 'test.csv'
layers = [['e_inf','w01','wp1','gamma1','w02','wp2','gamma2','thickness(nm)','level'], [str(i) for i in wavenumbers.tolist()]]
layers = list(chain(*layers))

train = pd.read_csv(path_train)
print(train.shape)
train = train.sample(frac=1, random_state=2)
rows, cols = train.shape

train1 = train.iloc[:rows - 54000,:]
train1 = train1.values
train1 = pd.DataFrame(data=train1,columns=layers)
train1.to_csv('train1_parameter.csv', index_label='id')

print("train file saved....")
val = train.iloc[rows - 54000:,:]
val = val.values
val = pd.DataFrame(data=val,columns=layers)
val.to_csv('val_parameter.csv', index_label='id')

(2754000, 360)
train file saved....


In [22]:
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
class Unet_1d(nn.Module):
    def __init__(self):
        super(Unet_1d, self).__init__()
        def CBR1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers +=[nn.Conv1d(in_channels=in_channels, out_channels=out_channels,
                               kernel_size=kernel_size, stride=stride, padding=padding,
                               bias=bias)]
            nn.init.xavier_uniform_(layers[0].weight)
            layers +=[nn.BatchNorm1d(num_features=out_channels)]
            layers +=[GELU()]
            layers +=[nn.Dropout(p=0.1)]
            
            cbr = nn.Sequential(*layers)
            
            return cbr
        def thickCN(in_channels, out_channels, kernel_size=3, stride=2, padding=0, bias=True):
            layers=[]
            layers +=[nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                                kernel_size=kernel_size, stride=stride, padding=padding,
                               bias=True)]
            layers +=[nn.BatchNorm1d(num_features=out_channels)]
            layers +=[GELU()]
            
            tcn = nn.Sequential(*layers)
            
            return tcn
        #input : batch_size x 351x1
        #trans_input: batch_size x 1 x351
        '''encoding'''
        self.enc1_1= CBR1d(1,64) #1x64x351
        self.enc1_2= CBR1d(64, 64) #1x64x351
        
        self.pool1=nn.MaxPool1d(kernel_size=3)#1x64x117
        
        self.enc2_1= CBR1d(64,128) #1x128x117
        self.enc2_2= CBR1d(128, 128) #1x128x117
        
        self.pool2=nn.MaxPool1d(kernel_size=3)#1x128x39
        
        self.enc3_1= CBR1d(128,256) #1x256x39
        self.enc3_2= CBR1d(256, 256) #1x256x39
        
        self.pool3=nn.MaxPool1d(kernel_size=3)#1x256x13
        '''decoding'''
        #1x256x13
        self.dec4_1=CBR1d(256,512)#1x512x13
        self.dec4_2=CBR1d(512,256)#1x256x13
        
        self.unpool3=nn.ConvTranspose1d(256, 256, kernel_size=3, stride=3, bias=True) #1x256x39
        
        self.dec3_2=CBR1d(512,256) #1x256x39
        self.dec3_1=CBR1d(256, 128) #1x128x39
        
        self.unpool2=nn.ConvTranspose1d(128, 128, kernel_size=3, stride=3, bias=True) #1x128x117
        
        self.dec2_2=CBR1d(256, 128)#1x128x117
        self.dec2_1=CBR1d(128, 64)#1x64x117
        
        self.unpool1=nn.ConvTranspose1d(64, 64, kernel_size=3, stride=3, bias=True) #1x64x351
        
        self.dec1_2=CBR1d(128, 64) #1x64x351
        self.dec1_1=CBR1d(64, 64) #1x64x351
        self.fc0=nn.Sequential(nn.Conv1d(64,2,kernel_size=1,stride=1,bias=True),nn.BatchNorm1d(2), GELU())
        #1x2x351
        self.fc1=nn.Sequential(nn.Linear(351*2,9))
        '''thickness network'''
        '''
        #1x201
        
        self.tpool2=nn.MaxPool1d(3,2)
        self.tunpool2=nn.ConvTranspose1d(256,128,kernel_size=3, stride=2, bias=True)
       
        self.fc2=nn.Sequential(nn.Linear(201*2,2))
        '''
        
    def forward(self, x):
        y=x[:,150:] #201x1
        #print(y.shape)
        x=x.view(x.shape[0],1,-1) #1000~3980
        y=y.view(y.shape[0],1,-1)#4000~8000
        '''nk_parameter_Unet'''
        x=self.enc1_1(x)
        enc1=self.enc1_2(x)
        enc1_pool=self.pool1(enc1)
        
        enc2=self.enc2_1(enc1_pool)
        enc2=self.enc2_2(enc2)
        enc2_pool=self.pool2(enc2)
        
        enc3=self.enc3_1(enc2_pool)
        enc3=self.enc3_2(enc3)
        enc3_pool=self.pool3(enc3)
        
        dec4=self.dec4_1(enc3_pool)
        dec4=self.dec4_2(dec4)
        
        dec3=self.unpool3(dec4)
        dec3=torch.cat([dec3,enc3], dim=1)
        dec3=self.dec3_2(dec3)
        dec3=self.dec3_1(dec3)
        
        dec2=self.unpool2(dec3)
        dec2=torch.cat([dec2, enc2], dim=1)
        dec2=self.dec2_2(dec2)
        dec2=self.dec2_1(dec2)
        
        dec1=self.unpool1(dec2)
        dec1=torch.cat([dec1, enc1], dim=1)
        dec1=self.dec1_2(dec1)
        dec1=self.dec1_1(dec1)
        
        out1=self.fc0(dec1)
        out1=out1.view(out1.shape[0], out1.size(1)*out1.size(2))
        out1=self.fc1(out1) #nk parameter
        '''thickness, level_CNN'''
        #1x201
        '''
        tenc1=self.enc1_1(y) 
        tenc1=self.enc1_2(tenc1) #1x64x201
        
        tenc2=self.pool1(tenc1) #1x64x67
        tenc2=self.enc2_1(tenc2) #1x128x67
        tenc2=self.enc2_2(tenc2) #1x128x67
        
        tenc3=self.tpool2(tenc2) #1x128x33
        tenc3=self.enc3_1(tenc3) #1x256x33
        tenc3=self.enc3_2(tenc3) #1x256x33
        
        tdec4=self.dec4_1(tenc3)
        tdec4=self.dec4_2(tdec4) #1x256x33
        
        tdec3=self.tunpool2(tenc3) #1x128x67
        tdec2=torch.cat([tdec3,tenc2], dim=1)#1x256x67
        tdec2=self.dec2_2(tdec2)#1x128x67
        tdec2=self.dec2_1(tdec2)#1x64x67
        
        tdec2=self.upool1(tdec2)#1x64x201
        tdec1=torch.cat([tdec2,tenc1], dim=1) #1x128x201
        tdec1=self.dec1_2(tdec1) #1x64x201
        tdec1=self.dec1_1(tdec1) #1x64x201
        
        out2=self.fc0(tdec1) #1x2x201
        out2=out2.view(out2.shape[0], out2.size(1)*out2.size(2)) #1x2*201
        out2=self.fc2(out2)
        '''

        return out1

def get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps, num_cycles=1.0, last_epoch=-1
):
    """ Create a schedule with a learning rate that decreases following the
    values of the cosine function with several hard restarts, after a warmup
    period during which it increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        if progress >= 1.0:
            return 0.0
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * ((float(num_cycles) * progress) % 1.0))))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

In [5]:
# 새로 만든 train/ val 모델 학습 데이터 경로를 설정.
train_path = 'train1_parameter.csv'#레이어를 같게 두고 풀때
val_path = 'val_parameter.csv'

class PandasDataset(Dataset):
    def __init__(self, path):
        super(PandasDataset, self).__init__()
        train = pd.read_csv(path).iloc[:,1:]
        self.train_X, self.train_Y = train.iloc[:,9:], train.iloc[:,0:9]
        self.tmp_x , self.tmp_y = self.train_X.values, self.train_Y.values
    
    def __len__(self):
        return len(self.train_X)

    def __getitem__(self, idx):
        return {
            'X':torch.from_numpy(self.tmp_x)[idx],
            'Y':torch.from_numpy(self.tmp_y)[idx]
        }

batch_size = 2048
            
train_dataset = PandasDataset(train_path)
train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, num_workers=0)#card에 data size/batch size만큼 monunt

val_dataset = PandasDataset(val_path)
val_loader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True, num_workers=0) 

# Model Picture

In [11]:
from torchviz import make_dot
import torch.onnx

In [18]:
model = Unet_1d()
dummy_data=torch.empty(1, 351, dtype=torch.float32)
torch.onnx.export(model, dummy_data, 'Unet_model.onnx')
model = model.to(device) # 모델을 GPU 메모리에 올림.

In [19]:
lr = 1e-03
adam_epsilon = 1e-06
epochs = 100
warmup_step = 2000
loss_fn1 = nn.MSELoss()
loss_fn2 = nn.L1Loss()

In [20]:
print(len(train_loader))

1319


In [23]:
"""
모델 학습
"""

total_step = len(train_loader) * epochs
print(f"Total step is....{total_step}") # 모델이 학습하는 전체 step 계산.

# 옵티마이저와 스케줄러의 파라미터들을 정의.

# no_decay = ["bias", "LayerNorm.weight"] # decay하지 않을 영역 지정.
# optimizer_grouped_parameters = [
#     {
#         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#         "weight_decay": 0.0,
#     },
#     {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
# ]
# optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=adam_epsilon)
optimizer = Adam(model.parameters(), lr = lr, eps=adam_epsilon)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_step, num_training_steps=total_step
)

# train loss와 val loss, thick_loss 지정.
total_loss = 0.0
total_val_loss = 0.0
total_thick_loss=0.0
total_level_loss=0.0
total_parameter_loss=0.0
# 모델 이름을 위해서 변수 만듦.
version = time.localtime()[3:5]
curr_lr = lr

n_val_loss = 10000000. # 가장 낮은 validation loss를 저장하기 위해서 변수 설정.
train_loss_epoch=[]
thick_loss_epoch=[]
level_loss_epoch=[]
parameter_loss_epoch=[]
val_loss_epoch=[]

start = time.time() #학습시간
for epoch in range(epochs):
    total_loss = 0 
    total_val_loss = 0
    total_parameter_loss=0
    total_thik_loss=0
    total_level_loss=0
    for i, data in enumerate(tqdm(train_loader, desc='*********Train mode*******')):  # train 데이터를 부르고 학습.
        # forward pass
#         pred1,pred2 = model(data['X'].float().to(device))
        pred1 = model(data['X'].float().to(device))
        loss_parameter = loss_fn1(pred1[:,:7], data['Y'].float().to(device)[:,:7])
        loss_thick=loss_fn1(pred1[:,7], data['Y'].float().to(device)[:,7])
        loss_level=loss_fn1(pred1[:,8], data['Y'].float().to(device)[:,8])
        loss = loss_parameter + loss_thick + loss_level
        
        # backward pass
        optimizer.zero_grad() # optimizer 객체 사용해서 학습 가능한 가중치 변수에 대한 모든 변화도를 0으로 만듦
        loss.backward() 
        optimizer.step() # update optimizer params
        scheduler.step() # update scheduler params
        
        total_loss += loss.item()
        total_parameter_loss += loss_parameter.item()
        total_thick_loss +=loss_thick.item()
        total_level_loss +=loss_level.item()
        
    train_loss = total_loss / len(train_loader)
    parameter_loss = total_parameter_loss/ len(train_loader)
    thick_loss = total_thick_loss/len(train_loader)
    level_loss = total_level_loss/len(train_loader)
    train_loss_epoch.append(train_loss)
    parameter_loss_epoch.append(parameter_loss)
    thick_loss_epoch.append(thick_loss)
    level_loss_epoch.append(level_loss)
    print ("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, epochs, train_loss))
    print ("Epoch [{}/{}], Thick Loss: {:.4f}".format(epoch+1, epochs, thick_loss))

    # evaluation
    # validation 데이터를 부르고 epoch 마다 학습된 모델을 부르고 평가.
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(tqdm(val_loader, desc='*********Evaluation mode*******')):
            pred1 = model(data['X'].float().to(device))
            loss_val_para = loss_fn1(pred1[:,:7], data['Y'].float().to(device)[:,:7])
            loss_val_thick=loss_fn1(pred1[:,7], data['Y'].float().to(device)[:,7])
            loss_val_level=loss_fn1(pred1[:,8], data['Y'].float().to(device)[:,8])
            loss_val=loss_val_para + loss_val_thick
            total_val_loss += loss_val.item()

    val_loss = total_val_loss / len(val_loader)
    val_loss_epoch.append(val_loss)
    print ("Epoch [{}/{}], Eval Loss: {:.4f}".format(epoch+1, epochs, val_loss))

    
    # best model을 저장.
    if val_loss < n_val_loss:
        n_val_loss = val_loss
        torch.save(model.state_dict(), f'test_percent_unit_{batch_size}_{version}_{lr}_{epochs}_MAE.pth')
        print("Best Model saved......")

*********Train mode*******:   0%|                                                             | 0/1319 [00:00<?, ?it/s]

Total step is....131900


*********Evaluation mode*******:   4%|█▊                                                | 1/27 [00:00<00:03,  6.54it/s]

Epoch [1/100], Train Loss: 64398.3026
Epoch [1/100], Thick Loss: 197.7059


*********Train mode*******:   0%|                                                             | 0/1319 [00:00<?, ?it/s]

...

Epoch [99/100], Eval Loss: 1079.5592
Best Model saved......


*********Evaluation mode*******:   4%|█▊                                                | 1/27 [00:00<00:03,  6.75it/s]

Epoch [100/100], Train Loss: 1054.0641
Epoch [100/100], Thick Loss: 1802.8349


*********Evaluation mode*******: 100%|█████████████████████████████████████████████████| 27/27 [00:03<00:00,  7.21it/s]

Epoch [100/100], Eval Loss: 1078.3152
Best Model saved......


In [6]:
# 4개의 layer_1, layer_2, layer_3, layer_4의 두께를 구하는 방법이라 마지막 Dense unit을 1로
# 해서 각각의 두께를 구하는 모델을 구성해보았으나 결과는 좋지 않음.
# 특별한 탐색적 자료 분석 없음.
"""
- Modeling

Learning rate : 다양한 learning rate 스케줄링. Warmup 스케줄링이 train, val loss 줄이는데 가장 효과적.
Activation function : ReLU, ELU, CeLU, GeLU 사용. 음의 영역에서는 0의 값을 가지는 relu 대신 ELU, CeLU, GeLU등을 사용. relu보다는 효과적.
Optimizer : adam, adamW.
Batch size : batch size는 10000으로 시작하였으나 10000보다 작은 값에서 효과적.
Hidden node, layer depth : node 수는 Nh = Ns/(a * (Ni + No))에 따라 초기 모델링에서는 약 2500개 정도로 하였으나 실험적으로 많은
node가 train, val loss 줄이는데 효과적.
(Nh : number of hidden , Ni : Number of input, No : Number of output, a : 2 - 10)
Loss funciotion : L1 Loss
BatchNorm : 모든 layer에 batchNorm 적용
MLP Model : 단순한 MLP 모델로는 한계를 느껴 MLP의 복잡성을 증가시키는 방향으로 설계
        (1) 노드 수를 증가시켰다 줄이는 방식(UP-block, DOWN-block)
        (2) Down-block에서 skip connection과 layer norm 적용
"""

'\n- Modeling\n\nLearning rate : 다양한 learning rate 스케줄링. Warmup 스케줄링이 train, val loss 줄이는데 가장 효과적.\nActivation function : ReLU, ELU, CeLU, GeLU 사용. 음의 영역에서는 0의 값을 가지는 relu 대신 ELU, CeLU, GeLU등을 사용. relu보다는 효과적.\nOptimizer : adam, adamW.\nBatch size : batch size는 10000으로 시작하였으나 10000보다 작은 값에서 효과적.\nHidden node, layer depth : node 수는 Nh = Ns/(a * (Ni + No))에 따라 초기 모델링에서는 약 2500개 정도로 하였으나 실험적으로 많은\nnode가 train, val loss 줄이는데 효과적.\n(Nh : number of hidden , Ni : Number of input, No : Number of output, a : 2 - 10)\nLoss funciotion : L1 Loss\nBatchNorm : 모든 layer에 batchNorm 적용\nMLP Model : 단순한 MLP 모델로는 한계를 느껴 MLP의 복잡성을 증가시키는 방향으로 설계\n        (1) 노드 수를 증가시켰다 줄이는 방식(UP-block, DOWN-block)\n        (2) Down-block에서 skip connection과 layer norm 적용\n'

In [32]:
loss_array=np.array(thick_loss_epoch)
loss_array2=np.array(val_loss_epoch)
loss_array3=np.array(level_loss_epoch)
loss_array4=np.array(train_loss_epoch)
thick_loss_data=pd.DataFrame(loss_array)
val_loss_data=pd.DataFrame(loss_array2)
level_loss_data=pd.DataFrame(loss_array3)
train_loss_data=pd.DataFrame(loss_array4)
thick_loss_data.to_csv(f'MAE_thick_same_layer_percent_unit_dropout=_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')#level을 동일하게 놔두면 평균 1nm까지 오차가 줄어든다.
val_loss_data.to_csv(f'MAE_val_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')
level_loss_data.to_csv(f'MAE_level_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')
train_loss_data.to_csv(f'MAE_train_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')

In [33]:
thick_loss_data.tail()

,0
95,0.000645
96,0.000633
97,0.000628
98,0.000626
99,0.000625


In [13]:
level_loss_data.tail()

,0
95,0.000416
96,0.000406
97,0.000394
98,0.000396
99,0.000392


In [24]:
test_file='R_mir_para.xlsx'
test=pd.read_excel(test_file)
test

,Unnamed: 0,1000,1020,1040,1060,1080,1100,1120,1140,1160,...,7820,7840,7860,7880,7900,7920,7940,7960,7980,8000
0,0,0.43274,0.45783,0.52730,0.59830,0.65145,0.67806,0.60875,0.53671,0.51018,...,0.12143,0.12246,0.12335,0.12420,0.12482,0.12569,0.12661,0.12747,0.12831,0.12928
1,1,0.43904,0.45938,0.52918,0.60372,0.66068,0.69203,0.62289,0.54710,0.51771,...,0.12842,0.12927,0.13033,0.13101,0.13195,0.13304,0.13423,0.13530,0.13612,0.13703
2,2,0.44766,0.47016,0.53749,0.60886,0.66225,0.68836,0.61643,0.54579,0.51949,...,0.11447,0.11525,0.11606,0.11677,0.11785,0.11863,0.11939,0.12046,0.12160,0.12265
3,3,0.44381,0.46831,0.53629,0.60635,0.65804,0.68132,0.60812,0.53913,0.51423,...,0.11003,0.11062,0.11120,0.11175,0.11260,0.11364,0.11464,0.11543,0.11617,0.11727
4,4,0.45495,0.47851,0.54659,0.61701,0.66936,0.69344,0.62036,0.55081,0.52515,...,0.11491,0.11568,0.11647,0.11699,0.11763,0.11808,0.11911,0.12006,0.12088,0.12158
5,5,0.44943,0.47328,0.54287,0.61628,0.67115,0.69813,0.62540,0.55229,0.52505,...,0.11952,0.12054,0.12127,0.12207,0.12293,0.12415,0.12509,0.12588,0.12685,0.12778


In [25]:
"""
모델 테스트
"""

test_model = Unet_1d()

# test 파일 경로 및 test 데이터 로드
path_test = 'R_mir_para.xlsx'
#path_test='val_same_layer_level.csv'
class TestDataset(Dataset):
    def __init__(self, path_test):
        super(TestDataset, self).__init__()
        test = pd.read_excel(path_test)
        self.test_X = test.iloc[:,1:]
        self.tmp_x = self.test_X.values
    
    def __len__(self):
        return len(self.test_X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.tmp_x)[idx]
    
test_data = TestDataset(path_test)
test_loader = DataLoader(test_data, batch_size=6,pin_memory=True, num_workers=0)

In [26]:
# 모델에 학습된 가중치를 업로드.
weights = torch.load(f'test_percent_unit_{batch_size}_{version}_{lr}_{epochs}_MAE.pth', map_location='cuda:0')
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()
test_time=time.time()
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        outputs = test_model(data.float())
    print('Test Time : ',time.time()-test_time)
pred_test = outputs

Test Time :  0.015620946884155273


In [27]:
#test_layer=[[str(i) for i in np.arange(1000,8100,100).tolist()],[str(i) for i in np.arange(1000,8100,100).tolist()],['layer(nm)','level(%)']]
test_layer=['e_inf','w01','wp1','gamma1','w02','wp2','gamma2','thickness(nm)','level']
# test_layer=list(chain(*test_layer))

In [70]:
#pred_test=torch.cat([pred_test[0],pred_test[1]],dim=1)

In [28]:
sub=pred_test.cpu().numpy()
sub=pd.DataFrame(data=sub, columns=test_layer)
sub.to_csv('result_mir_para.csv',index='id')

In [29]:
sub

,e_inf,w01,wp1,gamma1,w02,wp2,gamma2,thickness(nm),level
0,2.035898,1489.958496,631.394592,11.681986,1431.354248,827.135437,25.065491,268.843842,3.024388
1,1.900953,1423.363159,535.380859,10.937189,1354.855591,930.819641,12.952363,280.472351,-1.113766
2,2.249514,1330.242920,641.334900,19.546684,1193.747437,842.930969,36.449097,260.384918,2.633865
3,2.121203,1416.536255,503.962372,3.935621,1355.837646,815.849548,14.707232,263.487823,0.717771
4,2.283021,1400.824341,657.795959,25.116703,1111.161621,844.209167,22.601675,258.284851,2.731843
5,2.276467,1407.800659,545.293335,1.055789,1385.297485,818.026428,12.630845,257.684845,3.082365
